# Green loss calculation
This material is originally developed for ACE 592 Big Data - Empirical Economics written on 8/03/2016 by Akito Kamei, Johnathan Rush, and Peter Christensen. The initial material was modifed for a CyberGIS workshop, and more recently has been modified to run outside of the ROGER supercomputer.

### Objective:
* Read world map (shapefile)
* Read green loss satellite file (tiff file)
* Plot data

### Successful outcome:
* Calculate green loss by year

#### Information
Jupyter Notebooks autosave; check the text next to the file name at the top of the screen to see if it has saved before you close the browser tab. You can also go to the File menu and choose Save and Checkpoint, which gives you a place to revert back to if you need it.

If you are done running the notebook, go to File, then Close and Halt to free up system resources for other users. This will clear out all the variables and memory, but any output from the cells you have run will still be displayed. You can run your cells again from the beginning to re-populate variables.

## Step 1: Install packages 

In [ ]:
ipak <- function(pkg){
new.pkg <- pkg[!(pkg %in% installed.packages()[, "Package"])]
if (length(new.pkg)) 
    install.packages(new.pkg, dependencies = TRUE)
sapply(pkg, require, character.only = TRUE)
}

# usage
packages <- c("rgdal", "sp", "raster",  "gdalUtils", "foreach", "doSNOW", "rgeos")
ipak(packages)

## Step 2: Load the world map polygon data

In [ ]:
# Check your current working directory
getwd()

In [ ]:
# List the contents of our current working directory
list.files()

In [ ]:
# List the contents of the `basemap` subdirectory
list.files("basemap")

In [ ]:
#Read the world map shapefile that is inside of the basemap subdirectory
world=readOGR("basemap",layer="ne_110m_admin_0_countries")

### What does the readOGR function do?

In [ ]:
#Include a question mark and then a function name to pop up a help window
?readOGR

In [ ]:
# You can print an object, or just enter its name to get some basic information about it.
print(world)

In [ ]:
# Display the contents of the shapefile's attribute tabe
world@data

In [ ]:
# You can also get information on a class of objects:
?SpatialPolygonsDataFrame

## Step 3: Green loss calculation
We want to find out how much green loss there has been by country, by year. It's good to develop your approach with a small data set to start with so you can rapidly test it.
We will use the east African country of Guinea Bissau.

In [ ]:
world_Guinea_Bissau <- subset(world, sovereignt=="Guinea Bissau")

In [ ]:
world_Guinea_Bissau@data[,"sovereignt"]

In [ ]:
# Create a directory to hold loss year rasters, if it doesn't already exist
dir.create(file.path(getwd(), "lossyear"), showWarnings = FALSE)

In [ ]:
# Download a raster file with the green loss year encoded for each cell
# This one file is 10x10 degrees, and covers all of Guinea Bissau
download.file("https://storage.googleapis.com/earthenginepartners-hansen/GFC2015/Hansen_GFC2015_lossyear_20N_020W.tif", "lossyear/Hansen_GFC2015_lossyear_20N_020W.tif" )

In [ ]:
#### Find the raster that overlays Guinea-Bissau
# Read year of green loss satellite file (tiff file)
lossyear_gb <- raster("lossyear/Hansen_GFC2015_lossyear_20N_020W.tif")

#### Display the metadata for the raster file we just loaded
If you look at a map of Guinea Bissau, you'll see that the country fits entirely within the boundaries expressed by the Corner Coordinates of the raster below.

`gdalinfo` needs a filename to operate on, so we are getting the `name` slot of the `file` slot of the RasterLayer object we just made. Of course, we could have given it `"Hansen_GFC2015_lossyear_20N_020W.tif"`, too.

In [ ]:
print(gdalinfo(lossyear_gb@file@name))

#### Make map of Guinea-Bissau region

In [ ]:
# We start with the raster file, which will set the extent of the map
plot(lossyear_gb, add=FALSE)
plot(world, border="red", lwd = 2, add=TRUE)
plot(world_Guinea_Bissau, border="black", lwd = 4, add=TRUE)

## Summarize the green cover loss year in Guinea Bissau
Often, the approach used for this is the `extract` command in the Raster package. However, the performance of `extract` is poor, and uses an enormous amount of memory to allocate the extracted values.

The method below saves computation time by first cropping the raster to the bounding box of the Guinea Bissau polygon, then to the boundaries of the actual polygon, before extracting values. Often, sending smaller input files to a function will speed up the operation, even if many of the values in a large file _should_ be ignored.

The ```ext_gb``` object is a vector of every value extracted from the raster. These are then tabulated so that there is a count of every unique value. Finally, the tabulated summary is transformed into a data frame object.

In [ ]:
time1 <- proc.time() #start timer
startTimeGB <- time1
clip1_gb <- crop(lossyear_gb, extent(world_Guinea_Bissau)) #crop to extent of polygon
cat("crop 1:","\n"); proc.time() - time1

time1 <- proc.time() #start timer
clip2_gb <- rasterize(world_Guinea_Bissau, clip1_gb, mask=TRUE) #crops to polygon edge & converts to raster
cat("crop 2:","\n"); proc.time() - time1

time1 <- proc.time() #start timer
ext_gb <- getValues(clip2_gb) #much faster than extract
cat("getValues:","\n"); proc.time() - time1

time1 <- proc.time() #start timer
tab_gb <-table(ext_gb) #tabulates the values of the raster in the polygon
cat("tabulate:","\n"); proc.time() - time1

time1 <- proc.time() #start timer
mat_gb <- as.data.frame(tab_gb)
cat("as.data.frame(tab):","\n"); proc.time() - time1

timeGB <- proc.time() - startTimeGB
cat("Total time to extract raster cell values in Guinea Bissau:\n")
print(summary(timeGB))

#### Take a look at the summarized results data frame


In [ ]:
mat_gb

### Plot the summarized data frame as a bar chart

##### How to make a bar plot?
We'll check the help in the cell below. There, we've used the :: symbol to indicate which specific library's barplot function we want to use.

The `raster` library overrides and extends the built-in `barplot` function from the `graphics` library. So if we just ran `?barplot`, we'd get information on using `raster`'s version. However, there is better information on the basic capabilities of the barplot function in the `graphics` library's help.

In [ ]:
?graphics::barplot

In [ ]:
#Using values from position 2 on will exclude the first row - the 0 code for no loss.
# The 0 code is much more common than any other, and would make the plot hard to read.
GBlossyears <- mat_gb$Freq[2:length(mat_gb$Freq)]
GBlosscodes <- mat_gb$ext_gb[2:length(mat_gb$ext_gb)]
#Label setting
GBlosscodes <- factor(GBlosscodes,
levels = c("1","2","3","4","5","6","7","8","9","10","11","12","13","14"),
labels = c("2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014"))

barplot(height = GBlossyears, names.arg = GBlosscodes, xlab = "Code for Year of Green Loss", beside=TRUE)

## Step 4: Larger regions, Memory Management, and Parallelization

# Indonesia

### Get a copy of Indonesia's boundaries

In [ ]:
world_Indonesia <- subset(world, sovereignt=="Indonesia")

### Download Indonesia's green loss rasters
These nine files are about 290 MB all together

In [ ]:
# Get a vector of the files covering Indonesia
ind_files <- c("Hansen_GFC2015_lossyear_00N_090E.tif", 
                 "Hansen_GFC2015_lossyear_00N_100E.tif", 
                 "Hansen_GFC2015_lossyear_00N_110E.tif", 
                 "Hansen_GFC2015_lossyear_00N_120E.tif", 
                 "Hansen_GFC2015_lossyear_10N_090E.tif", 
                 "Hansen_GFC2015_lossyear_10N_100E.tif", 
                 "Hansen_GFC2015_lossyear_10N_110E.tif", 
                 "Hansen_GFC2015_lossyear_10N_120E.tif", 
                 "Hansen_GFC2015_lossyear_10S_120E.tif")

In [ ]:
localpath <- "lossyear"
remotepath <- "https://storage.googleapis.com/earthenginepartners-hansen/GFC2015"
for(i in 1:length(ind_files)){
    file <- ind_files[i]
    cat("Downloading ", file, "\n")
    download.file(paste(remotepath, file, sep="/"), paste(localpath, file, sep="/"))
}

### Build a *virtual raster* for ```lossyear``` files in for the Indonesia region
One raster file covered the entire region of Guinea Bissau, but we will need to use several adjacent rasters to cover Indonesia.

With a virtual raster file, we can reference many files as a single object. Virtual rasters have the file extension ```.vrt```

In [ ]:
#setwd("/mnt/jhub/lossyear")
#Create a vector of file names for the Indonesia area raster files
ind_lossyear <- c("lossyear/Hansen_GFC2015_lossyear_00N_090E.tif", 
                 "lossyear/Hansen_GFC2015_lossyear_00N_100E.tif", 
                 "lossyear/Hansen_GFC2015_lossyear_00N_110E.tif", 
                 "lossyear/Hansen_GFC2015_lossyear_00N_120E.tif", 
                 "lossyear/Hansen_GFC2015_lossyear_10N_090E.tif", 
                 "lossyear/Hansen_GFC2015_lossyear_10N_100E.tif", 
                 "lossyear/Hansen_GFC2015_lossyear_10N_110E.tif", 
                 "lossyear/Hansen_GFC2015_lossyear_10N_120E.tif", 
                 "lossyear/Hansen_GFC2015_lossyear_10S_120E.tif")

# create a VRT out of the input files, store in your directory
vrtpath <- paste(getwd(), "indonesia.vrt", sep="/")
gdalbuildvrt(ind_lossyear, vrtpath)

### Read in a virtual raster that refers to the TIFF files covering Indonesia

In [ ]:
IndRasters <- raster(vrtpath)

In [ ]:
# Two ways of getting information on the rasters we've loaded:
print(IndRasters) #print the default information for the raster object
print(gdalinfo(IndRasters@file@name)) #print the results of gdalinfo on the filename

### Plot the loss rasters with the national boundaries on top

In [ ]:
time1 <- proc.time() #start timer

plot(IndRasters, add=FALSE)
plot(world, border="red", lwd = 1, add=TRUE)
plot(world_Indonesia, border="black", lwd = 2, add=TRUE)

time2 <- proc.time() #end timer
cat("Plotting time for Indonesia:","\n")
summary(time2 - time1)

## Extracting raster values from Indonesia
We established earlier that extracting values from Indonesia the usual way would either take too long, or use too much memory.

One data parallelism strategy is called "chunking." Breaking the operations into smaller chunks can let you use more processors at once, as well as better manage how much memory you use.

Below, we will divide Indonesia into smaller pieces and summarize the extracted values from each one. By keeping only what we really want for each chunk - the summary of unique values - we can discard the array of every pixel value from each chunk as soon as we're done summarizing it. This will reduce the amount of memory we need to use at any one time.

After chunking, we will learn how to process many of the chunks at once to speed up computation.

#### How many chunks?
There is a little overhead in terms of storage and communication for every chunk you make, so you don't want to split up the source data _too_ much. One way to estimate if we are hitting the right balance is to take a look at the plotted map after we have the chunks. Better methods include checking the number of pixels in a chunk, the memory used by the values extracted for a chunk, or the ratio of a chunk's area to the area of a region we have already calculated (like Guinea Bissau).

In [ ]:
time1 <- proc.time() #start timer

# Get extent for country
ind_yext <- world_Indonesia@bbox["y","max"] - world_Indonesia@bbox["y","min"]
ind_xext <- world_Indonesia@bbox["x","max"] - world_Indonesia@bbox["x","min"]
cat("Indonesia longitude extent: ", ind_xext, "; latitude extent: ", ind_yext)

# Divide lat/long extents by U/V increments to create new polygons
# Here, we've manually chosen divisions, but you can imagine setting a maximum width for these
udiv = 45; vdiv = 15;

# Extent in map units for each rectangle
ind_yint <- ind_yext / vdiv
ind_xint <- ind_xext / udiv

# What is the size of each rectangle?
cat("\nSubdivision rectangle width:  ", ind_xint)
cat("\nSubdivision rectangle height: ", ind_yint)

#initialize an empty list with the length to hold all the grid cells we will create
polys <- vector("list", udiv*vdiv) #make a vector of type list, with a length of udiv * vdiv

#loop
for (u in 1:udiv){
    for (v in 1:vdiv){     
        #store the calculated extents as polygons in an indexed array
        polys[[(u-1)*vdiv+v]] <- as(extent(world_Indonesia@bbox["x","min"] + ind_xint * (u-1),
                                           world_Indonesia@bbox["x","min"] + ind_xint * u, 
                                           world_Indonesia@bbox["y","min"] + ind_yint * (v-1),
                                           world_Indonesia@bbox["y","min"] + ind_yint * v),
                                     'SpatialPolygons')
    }
}

#merge polygons - this calls the bind function on each polygon to make them a single object
merged_polys <- do.call(bind, polys) 

#set the projection of these new shapes to the same as is used for our Indonesia shape
projection(merged_polys) <- projection(world_Indonesia)

cat("\nSubdivision rectangle creation time","\n"); proc.time() - time1

#### Plot the newly created cells to check that they look correct

In [ ]:
plot(merged_polys, col='red', axes=TRUE)
plot(world_Indonesia, col=rgb(1,1,1,0.5), add=TRUE)

## Intersected result

In [ ]:
#Intersected chunks with country shape
chunks_Ind <- intersect(world_Indonesia, merged_polys)
plot(chunks_Ind)

#### How many cells overlapped with Indonesia?

In [ ]:
length(chunks_Ind)

### Extract values from Indonesia chunks - Serial version
We will extract the loss year values from small regions of Indonesia, one at a time. This solves the problem of loading too much data into memory.  
However, there are enough chunks that this would take a long time. We'll stop early, and you can extrapolate out how long it would take to extract the values for the whole country.

In [ ]:
shapecount <- length(chunks_Ind) # total number of chunks

endcount <- 10 # choose a number less than shapecount for testing
mat <- vector("list", endcount)

extractor <- function(p){
    clip1_Ind <- crop(IndRasters, extent(chunks_Ind[p,])) #crop to extent of polygon
    clip2_Ind <- rasterize(chunks_Ind[p,], clip1_Ind, mask=TRUE)
    ext <- getValues(clip2_Ind) #much faster than raster::extract
    print(ext)
    tab<-table(ext) #tabulates the values of the raster in the polygon
    print(tab)
}

starttimeInd <- proc.time() #begin time

#apply (run) the extractor function for each chunk of Indonesia, up to the endcount limit
tablesInd <- lapply(1:endcount, function(p) extractor(p)) 
    
dfInd <- do.call(cbind, tablesInd) #call the cbind (combine by columns) function on each table in the `tablesInd` list
endtimeInd <- proc.time() #end time

cat("Time to extract values from ", endcount, " polygons:\n")
print(summary(endtimeInd - starttimeInd))

cat("\nEstimated total time for serial extraction of all Indonesia values:\n")
cat((endtimeInd - starttimeInd)[[3]] * length(chunks_Ind) / endcount / 60, " minutes")

#dfInd is a data frame with a column for every table from the tablesInd list.
# Sum across each row to get the frequency of each raster value
sumInd <- as.data.frame(rowSums(dfInd)) 
colnames(sumInd) <- "Freq" #assign the name Freq to the column in the data frame to be consistent with Guinea Bissau method
sumInd

### Extract values from Indonesia chunks - Parallel version
As above, but using `foreach` instead of `lapply`. 

In [ ]:
shapecount <- length(chunks_Ind) # total number of chunks

endcount <- 10 # use the same number as the above serial version to measure speedup
              # SET TO SHAPECOUNT to extract data for all of Indonesia
extractor <- function(p){1
    clip1_Ind <- crop(IndRasters, extent(chunks_Ind[p,])) #crop to extent of polygon
    clip2_Ind <- rasterize(chunks_Ind[p,], clip1_Ind, mask=TRUE)
    ext <- getValues(clip2_Ind) #much faster than raster::extract
    tab<-table(ext) #tabulates the values of the raster in the polygon
}

# Change `4` to however many processors your system has
cluster = makeCluster(4, type = "SOCK")
registerDoSNOW(cluster)
clusterExport(cluster, c("IndRasters","chunks_Ind", "extractor"))
ptm <- proc.time()    
results = foreach(n = 1:endcount, .combine = cbind) %dopar% {
     library(raster); 
     #Define function
     extractor(n)
}
#results
proc.time() - ptm 
stopCluster(cluster)

sumInd <- as.data.frame(rowSums(results))
colnames(sumInd) <- "Freq"
sumInd

## Question: How much did the parallel code speed up a sequential code?
Sp = Ts / Tp
- p = # of processors
- Ts = execution time of the sequential code
- Tp = execution time of the parallel code with p processors

In [ ]:
167.559 / 63.288